## Notation for solving the relocating problem in [Braverman et al. 2019]

### Part 1: The basics
#### *Parameters*
$r \in \mathbb{R}_+$: Number of regions 

$N \in \mathbb{R}_+$: Robotaxi fleet size

$\lambda \in \mathbb{R}^r_+$: A vector for riders' arrival rate (Poisson) across all regions <br>

$P \in \mathbb{R}^{r\times r}_+$: State transition matrix for a ride's trip, allowing $P_{ii}>0$

$\mu \in \mathbb{R}^{r\times r}_+$: Reciprocal of mean travel times <br> $1/\mu_{ij}$ is the mean travel time from region $i$ to region $j$ for a robotaxi <br>

[we can assume travel times from $i$ to $j$ for all robotaxies are i.i.d. exponential r.v. with mean $1/\mu_{ij}$]

#### *To solve*:
$Q \in \mathbb{R}^{r\times r}_+$: State transition matrix for relocating a robotaxi (when it is empty)

#### *Other variables*:
$E^{(N)}(t) \in \mathbb{Z}_+^{r\times r}, t\geq 0$: Empty-car distribution matrix at flet size $N$ and time step $t$, <br> $E^{(N)}_{ij}(t)$ = \# of empty cars going from $i$ to $j$ or stays in $i$ if $j=i$ <br>
$\bar{E}^{(N)}(t) = E^{(N)}(t)/N$

$F^{(N)}(t) \in \mathbb{Z}_+^{r\times r}, t\geq 0$: Occupied-car distribution matrix at flet size $N$ and time step $t$, <br> $F^{(N)}_{ij}(t)$ = \# of occupied cars going from $i$ to $j$ or stays in $i$ if $j=i$ <br>
$\bar{F}^{(N)}(t) = F^{(N)}(t)/N$ <br>
$\bar{E}^{(N)}(\infty), \bar{F}^{(N)}(\infty)$ are the stationary distribution of the $(\bar{E}^{(N)}, \bar{F}^{(N)})$ CTMC. <br>
This CTMC has a single recurrence class, so all regions are reachable from any other region.

$\mathcal{J} = \{(e,f)\in [0,1]^{r\times r}\times [0,1]^{rxr}: \sum_{i,j=1}^r (e_{ij}+f_{ij}) = 1\}$ 

This is the set of state space (empty-car percentage over all (stay+en route) x occupied-car percentage over all (stay+en route)); the total sum is 1 which is the entire fleet.

$A^{(N)}(\infty) \in [0,1]^r$ where $A_i^{(N)}(\infty) = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)>0)$

$\bar{a} \in [0,1]^r$ where $\bar{a}_i = \mathbb{P}(\bar{e}_{ii}>0)$

### Part 2: The optimization problem to solve an optimal static relocating policy

$c_{ij} > 0$: Reward for completing a trip from $i$ to $j$

$q \in \mathbb{R}^{rxr}$: $q_{ij}$'s are decision variables representing the static policy

$(\bar{e}, \bar{f}, \bar{a}) \in \mathcal{J}\times [0,1]^r$ <br>
$\bar{e} = \mathbb{E}[\bar{E}^{(N)}(\infty)], \bar{f} = \mathbb{E}[\bar{F}^{(N)}(\infty)], \bar{a} = A^{(N)}(\infty)$

$\bar{a}_i \lambda_i P_{ij}$: The rate that a ride (car w/ rider) which is from $i$ to $j$ arrives

$\bar{a}_i = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)>0)$

### Formulation \# 1

\begin{align}
\max_{q,\bar{e},\bar{f},\bar{a}}&\sum_{i=1}^r \sum_{j=1}^r \bar{a}_i\lambda_i P_{ij}c_{ij} \quad\bar{a}_i \lambda_i P_{ij}= \text{ the rate that a ride which is from $i$ to $j$ arrives}\\
\text{subject to } \bar{f}_{ij} &= \frac{1}{\mu_{ij}}\bar{a}_i\lambda_iP_{ij}\quad \forall i,j=1,\cdots,r\quad \text{Little's Law }L=W\lambda \text{ for occupied cars going from $i$ to $j$}\\
\bar{e}_{ij} &= \frac{1}{\mu_{ij}}q_{ij}\sum_{k=1}^r \mu_{ki}\bar{f}_{ki} \quad\forall 1 \leq i\neq j=1 \leq r\quad\text{Little's Law for empty cars cars going from $i$ to $j$}\\
&\text{RHS is the rate that empty cars start their trips}\\
\lambda_i \bar{a}_i &= \sum_{k=1,k\neq i}^r \mu_{ki}\bar{e}_{ki} + q_{ii}\sum_{k=1}^r\mu_{ki}\bar{f}_{ki} \quad\forall i=1,\cdots,r\quad\text{Car flow balance at region $i$}\\
(1- &\bar{a}_i)\bar{e}_{ii} = 0 \quad\forall i=1,\cdots,r\quad\text{Availability and empty car relation}\\
(\bar{e},&\bar{f}) \in \mathcal{J} \quad\text{ unit mass constraints }\\
q_{ij}&\geq 0,\quad \sum_{j=1}^r q_{ij} = 1,\quad 0\leq \bar{a}_{i}\leq 1ç
\end{align}
For car flow balance, LHS is total outflow rate (rider requests) from region $i$ (rider arrival rate x if empty car is available); RHS is total inflow rate (from other regions and ones that stays in $i$)<br>
For availability, $1-\bar{a}_i  = \mathbb{P}(\bar{E}^{(N)}_{ii}(\infty)=0)$, so $1-\bar{a}_i = 0$ *iff* $\bar{e}_{ii}>0$


#### Remark on Formulation \# 1 
From Section 3.2 of **Braverman et al. 2019**:
The fluid-based empty-car routing policy requires 2 important assumptions: <br>
1. The parameters remain constant over time
2. The system has reached equilibrium

Common solution: divide the day into periods of time where parameters are approximately constant w.r.t. each period.

### T-lookahead policy

For time window $[t, t+T]$:

\begin{align}
\max_{q,\bar{e},\bar{f},\bar{a}}\;\frac{1}{T}\int_0^T\bigg(\sum_{i=1}^r &\sum_{j=1}^r \bar{a}_i\lambda_i(t+u)\cdot P_{ij}(t+u)\cdot c_{ij}(t+u)\bigg)du\\
\text{subject to } \bar{f}_{ij} &= \frac{1}{T}\int_0^T\bigg(\frac{1}{\mu_{ij}(t+u)}\bar{a}_i\lambda_i(t+u)\cdot P_{ij}(t+u) \bigg)du
\quad\quad\forall i,j=1,\cdots,r\quad\quad \text{Little's Law }L=W\lambda \text{ for occupied cars going from $i$ to $j$}\\
\bar{e}_{ij} &= \frac{1}{T}\int_0^T\bigg(\frac{1}{\mu_{ij}(t+u)}q_{ij}\sum_{k=1}^r \mu_{ki}(t+u)\bar{f}_{ki}\bigg)du
\quad\forall 1 \leq i\neq j=1 \leq r\quad\quad\;\;\text{Little's Law }L=W\lambda \text{ for empty cars cars going from $i$ to $j$}\\
\frac{1}{T}\int_0^T\lambda_i(t+u) \bar{a}_idu &= \frac{1}{T}\int_0^T\bigg( \sum_{k=1,k\neq i}^r \mu_{ki}(t+u)\bar{e}_{ki} + q_{ii}\sum_{k=1}^r\mu_{ki}(t+u)\bar{f}_{ki}\bigg)du
\quad\forall i=1,\cdots,r\quad\text{Car flow balance at region $i$}\\
(1- &\bar{a}_i)\bar{e}_{ii} = 0 \quad\forall i=1,\cdots,r\quad\text{Availability and empty car relation}\\
(\bar{e},&\bar{f}) \in \mathcal{J} \quad\text{ unit mass constraints }\\
q_{ij}&\geq 0,\quad \sum_{j=1}^r q_{ij} = 1,\quad 0\leq \bar{a}_{i}\leq 1\quad\forall i=1,\cdots,r
\end{align}

### Discrete-time Lookahead policy
$K$ time blocks, each time block has length $\Delta$<br>
$$\begin{align*}
\max_{\bar{e}, \bar{f}, \bar{a}} \quad & \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\sum_{i,j=1}^r\bar{a}_i \lambda_i(t + k\Delta) P_{ij}(t + k\Delta) c_{ij}(t + k\Delta)\bigg)\Delta \\
\text{subject to} \quad 
& \bar{f}_{ij} = \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\frac{1}{\mu_{ij}(t + k\Delta)}\bar{a}_i\lambda_i(t + k\Delta) P_{ij}(t + k\Delta) \bigg) \Delta\\
& \bar{e}_{ij} \leq \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\frac{1}{\mu_{ij}(t + k\Delta)}\sum_{l=1}^r \mu_{li}(t + k\Delta)  \bar{f}_{li}\bigg)\Delta\\
& \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\sum_{\substack{j=1 \\ j \neq i}}^r \mu_{ji}(t + k\Delta) \bar{e}_{ji}\bigg)\Delta \leq \frac{1}{K\Delta}\sum_{k=0}^{K-1} \bigg(\bar{a}_i\lambda_i(t + k\Delta)\bigg)\Delta  \\
& \frac{1}{K\Delta}\sum_{k=0}^{K-1} \bigg(\bar{a}_i\lambda_i(t + k\Delta)\bigg)\Delta
\leq \frac{1}{K\Delta} \sum_{k=0}^{K-1}\bigg(\sum_{\substack{j=1 \\ j \neq i}}^r \mu_{ji}(t + k\Delta)\bar{e}_{ji} + \sum_{j=1}^r \mu_{ji}(t + k\Delta) \bar{f}_{ji}\bigg)\Delta \\
& \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\lambda_i(t + k\Delta)\bar{a}_i + \sum_{\substack{j=1 \\ j \neq i}}^r 
\frac{1}\mu_{ij}(t + k\Delta)\bar{e}_{ij}\bigg)\Delta \\
&\quad = \frac{1}{K\Delta} \sum_{k=0}^{K-1} \bigg(\sum_{\substack{j=1 \\ j \neq i}}^r  \mu_{ji}(t + k\Delta) \bar{e}_{ji} + \sum_{j=1}^r  \mu_{ji}(t + k\Delta) \bar{f}_{ji}\bigg)\Delta \\
& e_{ij}, f_{ij} \in [0, 1], \quad\sum_{i,j=1}^r e_{ij} + f_{ij} = 1\quad 0 \leq \bar{a}_i \leq 1
\end{align*}
$$